In [89]:
import sys
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt

sys.path.append('../../')
%load_ext autoreload
%autoreload 
from CostFunctions import CostFunction
from QEnvs.QEnvWave import QEnvWave
from Pulses import STA_pulses

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
proportional_weights = [-2, -1., 1.] # [gamma, beta, alpha] gamma < beta < alpha
integral_weights = [0, -0.02, 0]
derivative_weights = [0, 0, 25.]

num_qubits = len(proportional_weights)

time_max = 20
num_steps = 100
omega_max = 1
initial_state = qt.basis(3, 0)
target_state = qt.basis(3, 2)

pulses_worst = np.zeros((num_qubits - 1, num_steps))
pulses_middle = np.array([np.array([np.ones(num_steps//2), np.zeros(num_steps//2)]).flatten(), np.array([np.zeros(num_steps//2), np.ones(num_steps//2)]).flatten()])*omega_max*num_qubits/time_max
pulses_best = STA_pulses(num_qubits, time_max, num_steps, omega_max).T

pulses = np.array([pulses_worst, pulses_middle, pulses_best])
cases = ['No Transfer', 'Sequential Transfer', 'Optimal Transfer']

In [91]:
def solveForCostClass(costClass):
    env = QEnvWave(
                    num_qubits=num_qubits,
                    time_max=time_max,
                    num_steps=num_steps,
                    cost_function=costClass.costFunction)

    costs = []
    populations = []
    for i in range(len(pulses)):
        env._reset()
        for j in range(num_steps):
            env._step(pulses[i][:, j])
        costs.append(env.costs.T[:-1])
        populations.append(env.populations.T[:-1])

    return costs, populations, env.times

In [92]:
P_costClass = CostFunction(
    proportional_weights=proportional_weights)
P_costs, populations, times = solveForCostClass(P_costClass)

PI_costClass = CostFunction(
    proportional_weights=proportional_weights,
    integral_weights=integral_weights)
PI_costs, populations, times = solveForCostClass(PI_costClass)

PID_costClass = CostFunction(
    proportional_weights=proportional_weights,
    integral_weights=integral_weights,
    derivative_weights=derivative_weights)
PID_costs, populations, times = solveForCostClass(PID_costClass)

costs = [P_costs, PI_costs, PID_costs]
costs_labels = ['P', 'PI', 'PID']

In [93]:
# Save results
labels = ["Times", "Pulses", "Populations", "Costs"]
save = np.array([np.array(labels), times, cases, pulses, populations, costs_labels, costs])
np.save("Data/cost_functions.npy", save)

/tmp/ipykernel_7203/2403773862.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  save = np.array([np.array(labels), times, cases, pulses, populations, costs_labels, costs])
